In [1]:
import os
import pywt
import librosa
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import *

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import *
from tensorflow.keras.callbacks import EarlyStopping
import time


import warnings
warnings.filterwarnings('ignore')


2023-06-27 04:31:43.182840: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
path = os.getcwd() + "/audio_dataset/"
path

'/root/audio_dataset/'

In [3]:
TRAINING_DIR = path+"/train"
VALIDATION_DIR = path + "/val"
TEST_DIR = path + "/test"

print(TRAINING_DIR)
print(VALIDATION_DIR)
print(TEST_DIR)

/root/audio_dataset//train
/root/audio_dataset//val
/root/audio_dataset//test


In [4]:
def load_wav_16k_mono(filename):
    wav, sr = librosa.load(filename, sr = 16000, mono = True)
    sr = tf.cast(sr, dtype = tf.int64)
    return wav, sr


In [5]:
# audio_data = []
# labels = []
# sampling_rate = []
# file_names = []

# d = 0
# for file_name in Queen:
#     d += 1
#     audio, sr = load_wav_16k_mono(os.path.join(path + "\\Queen", file_name))
#     labels.append(1)
#     sampling_rate.append(sr)
#     file_names.append(file_name)
#     audio_data.append(audio)
#     if d == 1000:
#         break
# d = 0
# for file_name in QueenLess:
#     d += 1
#     audio, sr = load_wav_16k_mono(os.path.join(path + "\\Queenless", file_name))
#     labels.append(0)
#     sampling_rate.append(sr)
#     file_names.append(file_name)
#     audio_data.append(audio)
#     if d == 1000:
#         break

In [6]:
def data_loader(path):
    audio_data = []
    labels = []
    for folder in os.listdir(path):
        for file in os.listdir(os.path.join(path, folder)):
            audio, sr = load_wav_16k_mono(os.path.join(path, folder, file))
            audio_data.append(audio)
            labels.append(folder)
    return audio_data, labels

In [7]:
X_train, Y_train = data_loader(TRAINING_DIR)
X_val, Y_val = data_loader(VALIDATION_DIR)
X_test, Y_test = data_loader(TEST_DIR)

2023-06-27 04:31:47.560893: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22078 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:c1:00.0, compute capability: 8.9


In [8]:
print("There are {} training samples and {} training labels".format(len(X_train), len(Y_train)))
print("There are {} validation samples and {} validation labels".format(len(X_val), len(Y_val)))
print("There are {} testing samples and {} testing labels".format(len(X_test), len(Y_test)))

There are 14000 training samples and 14000 training labels
There are 2000 validation samples and 2000 validation labels
There are 4000 testing samples and 4000 testing labels


In [9]:
X_train = np.array(X_train)
X_val = np.array(X_val)
X_test = np.array(X_test)
Y_train = np.array(Y_train)
Y_val = np.array(Y_val)
Y_test = np.array(Y_test)

print("Shape of X_train: {}".format(X_train.shape))
print("Shape of X_val: {}".format(X_val.shape))
print("Shape of X_test: {}".format(X_test.shape))

Shape of X_train: (14000, 32000)
Shape of X_val: (2000, 32000)
Shape of X_test: (4000, 32000)


In [10]:
# Reshape the input data
X_train = X_train.reshape( (X_train.shape[0], X_train.shape[1], 1) )
X_val = X_val.reshape( (X_val.shape[0], X_val.shape[1], 1) )
X_test = X_test.reshape( (X_test.shape[0], X_test.shape[1], 1) )
Y_train = Y_train.reshape( (Y_train.shape[0], 1) )
Y_val = Y_val.reshape( (Y_val.shape[0], 1) )
Y_test = Y_test.reshape( (Y_test.shape[0], 1) )

print("Shape of X_train: {}".format(X_train.shape))
print("Shape of X_val: {}".format(X_val.shape))
print("Shape of X_test: {}".format(X_test.shape))

Shape of X_train: (14000, 32000, 1)
Shape of X_val: (2000, 32000, 1)
Shape of X_test: (4000, 32000, 1)


In [11]:
label_encoder = LabelEncoder()
Y_train = label_encoder.fit_transform(Y_train)
Y_val = label_encoder.fit_transform(Y_val)
Y_test = label_encoder.fit_transform(Y_test)

In [12]:
# shuffle the dataset
shuffle_index = np.random.permutation(len(X_train))
X_train, Y_train = X_train[shuffle_index], Y_train[shuffle_index]
shuffle_index = np.random.permutation(len(X_val))
X_val, Y_val = X_val[shuffle_index], Y_val[shuffle_index]
shuffle_index = np.random.permutation(len(X_test))
X_test, Y_test = X_test[shuffle_index], Y_test[shuffle_index]

In [13]:
model = Sequential([
    layers.Input(shape=(X_train.shape[1:])),
    layers.Conv1D(16, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(32, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(64, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(128, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(256, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Conv1D(512, 3, activation='relu'),
    layers.MaxPooling1D(2),
    layers.Flatten(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

In [14]:
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'],
             )
# tf.keras.losses.BinaryCrossentropy()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 31998, 16)         64        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 15999, 16)        0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 15997, 32)         1568      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 7998, 32)         0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 7996, 64)          6208      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 3998, 64)         0

In [15]:
earlystopper = EarlyStopping(
    monitor='val_loss',
    patience=20,
    restore_best_weights=True
)

In [23]:
epochs = 120
batch_size = 32
current_time = int(time.time())
stop_training_callback = earlystopper
history = model.fit(X_train, Y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_data=(X_val, Y_val), 
                    callbacks=[stop_training_callback])
print("Training completed in {} seconds.".format(int(time.time()-current_time)))

Epoch 1/120
438/438 [==============================] - 16s 35ms/step - loss: 0.4207 - accuracy: 0.8171 - val_loss: 0.4712 - val_accuracy: 0.7825
Epoch 2/120
438/438 [==============================] - 13s 30ms/step - loss: 0.3848 - accuracy: 0.8304 - val_loss: 0.5156 - val_accuracy: 0.7505
Epoch 3/120
438/438 [==============================] - 14s 31ms/step - loss: 0.3594 - accuracy: 0.8401 - val_loss: 0.4598 - val_accuracy: 0.7975
Epoch 4/120
438/438 [==============================] - 13s 30ms/step - loss: 0.3443 - accuracy: 0.8497 - val_loss: 0.4765 - val_accuracy: 0.7945
Epoch 5/120
438/438 [==============================] - 13s 30ms/step - loss: 0.3019 - accuracy: 0.8719 - val_loss: 0.5166 - val_accuracy: 0.7970
Epoch 6/120
438/438 [==============================] - 13s 30ms/step - loss: 0.2888 - accuracy: 0.8783 - val_loss: 0.5207 - val_accuracy: 0.7790
Epoch 7/120
438/438 [==============================] - 13s 30ms/step - loss: 0.2303 - accuracy: 0.9021 - val_loss: 0.5885 - val_ac

In [24]:
test_predict = model.predict(X_test)

125/125 [==============================] - 1s 5ms/step


In [25]:
show_test_predict = np.round(test_predict, 0)
show_test_predict = pd.DataFrame(show_test_predict)
print(show_test_predict)
print(len(show_test_predict))

        0
0     1.0
1     0.0
2     1.0
3     1.0
4     0.0
...   ...
3995  1.0
3996  1.0
3997  1.0
3998  1.0
3999  1.0

[4000 rows x 1 columns]
4000


In [26]:
from sklearn.metrics import f1_score
f1_score(Y_test, show_test_predict, average='macro')

0.8044591246903385

In [27]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(Y_test, show_test_predict)

In [28]:
cf

array([[1449,  551],
       [ 226, 1774]])

In [30]:
print(classification_report(Y_test, show_test_predict))

              precision    recall  f1-score   support

           0       0.87      0.72      0.79      2000
           1       0.76      0.89      0.82      2000

    accuracy                           0.81      4000
   macro avg       0.81      0.81      0.80      4000
weighted avg       0.81      0.81      0.80      4000

